In [ ]:
## ISSAM SALMI

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
import os

In [ ]:
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip

--2024-03-26 00:01:14--  https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.163.207, 142.251.167.207, 172.253.62.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.163.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘horse-or-human.zip.2’

horse-or-human.zip. 100%[===================>] 142.65M   134MB/s    in 1.1s    

2024-03-26 00:01:16 (134 MB/s) - ‘horse-or-human.zip.2’ saved [149574867/149574867]



In [ ]:
import zipfile

# Unzip the dataset
local_zip = './horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./horse-or-human')
zip_ref.close()

In [ ]:
import os

# Directory with our training horse pictures
train_horse_dir = os.path.join('./horse-or-human/horses')

# Directory with our training human pictures
train_human_dir = os.path.join('./horse-or-human/humans')

In [ ]:
train_horses_names = os.listdir(train_horse_dir)
train_humans_names = os.listdir(train_human_dir)

In [ ]:
train_humans_names[:2]

['human09-05.png', 'human15-17.png']

In [ ]:
train_horses_names[:2]

['horse14-8.png', 'horse43-2.png']

In [ ]:
print(f"total training horses images = {len(os.listdir(train_horse_dir))}")
print(f"total training humans images = {len(os.listdir(train_human_dir))}")

total training horses images = 500
total training humans images = 527


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image  as mpimg
nrows = 4
ncols = 4
pic_index = 0

In [ ]:
# Set up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

pic_index += 8
next_horse_pix = [os.path.join(train_horse_dir, fname)
                for fname in train_horses_names[pic_index-8:pic_index]]
next_human_pix = [os.path.join(train_human_dir, fname)
                for fname in train_humans_names[pic_index-8:pic_index]]

for i, img_path in enumerate(next_horse_pix+next_human_pix):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # Don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

In [ ]:
# Building a Small Model from Scratch

In [ ]:
model = tf.keras.Sequential([
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3, 3), activation = 'relu', input_shape = (300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # This is the second convolution
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    # This is the Third convolution
    tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    #----
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 149, 149, 16)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 73, 73, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 35, 35, 64)        0

In [ ]:
from tensorflow.keras.optimizers import RMSprop

In [ ]:
model.compile(loss = 'binary_crossentropy',
                        optimizer = RMSprop(learning_rate=0.001),
                        metrics = ['accuracy'])

### **Data Preprocessing**



Next step is to set up the data generators that will read pictures in the source folders, convert them to float32 tensors, and feed them (with their labels) to the model. You'll have one generator for the training images and one for the validation images. These generators will yield batches of images of size 300x300 and their labels (binary).

As you may already know, data that goes into neural networks should usually be normalized in some way to make it more amenable to processing by the network (i.e. It is uncommon to feed raw pixels into a ConvNet.) In this case, you will preprocess the images by normalizing the pixel values to be in the [0, 1] range (originally all values are in the [0, 255] range).

In Keras, this can be done via the keras.preprocessing.image.ImageDataGenerator class using the rescale parameter. This ImageDataGenerator class allows you to instantiate generators of augmented image batches (and their labels) via .flow(data, labels) or .flow_from_directory(directory).

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen   = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    './horse-or-human',
    target_size = (300, 300),
    batch_size  = 128,
    class_mode  = 'binary'
)

Found 1027 images belonging to 2 classes.


In [ ]:
history = model.fit(train_generator,
                    steps_per_epoch = 8,
                    epochs = 15,
                    verbose = 1)

Epoch 1/15
8/8 [==============================] - 93s 11s/step - loss: 1.6716 - accuracy: 0.5228
Epoch 2/15
8/8 [==============================] - 89s 11s/step - loss: 0.6936 - accuracy: 0.5818
Epoch 3/15
8/8 [==============================] - 88s 12s/step - loss: 1.0851 - accuracy: 0.6707
Epoch 4/15
8/8 [==============================] - 89s 11s/step - loss: 0.6239 - accuracy: 0.6296
Epoch 5/15
8/8 [==============================] - 89s 11s/step - loss: 0.5293 - accuracy: 0.7430
Epoch 6/15
8/8 [==============================] - 89s 11s/step - loss: 0.6260 - accuracy: 0.7875
Epoch 7/15
8/8 [==============================] - 89s 11s/step - loss: 0.3365 - accuracy: 0.8776
Epoch 8/15
8/8 [==============================] - 89s 11s/step - loss: 0.2839 - accuracy: 0.8954
Epoch 10/15
8/8 [==============================] - 89s 11s/step - loss: 0.1234 - accuracy: 0.9466
Epoch 11/15
8/8 [==============================] - 90s 11s/step - loss: 0.2251 - accuracy: 0.9210
Epoch 12/15
8/8 [===========

In [ ]:
import numpy as np
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array

In [ ]:
uploaded = files.upload()

for fn in uploaded.keys() :
  path = '/content/' + fn
  imge = load_img(path, target_size = (300, 300))
  x    = img_to_array(imge)
  x   /= 255
  x    = np.expand_dims(x, axis = 0)
  images  = np.vstack([x])
  classes = model.predict(images, batch_size = 10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")

Saving pexels-samad-ismayilov-1270076.jpg to pexels-samad-ismayilov-1270076.jpg
1/1 [==============================] - 0s 47ms/step
[0.99999994]
pexels-samad-ismayilov-1270076.jpg is a human
